# Lymphoid_No4-RV-step1_edgeR_DEgenes

## Import required modules

In [1]:
import os
import anndata
import logging
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import leidenalg
import csv
from pandas import Timestamp
import re
import matplotlib.pyplot as plt
from scipy.stats import rankdata 
import scipy
import harmonypy as hm
import random
from scipy.stats.stats import pearsonr   
import scipy.sparse
from SCCAF import *

In [2]:
sc.logging.print_header()
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=140, color_map='RdBu_r')

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.
scanpy==1.6.0 anndata==0.7.4 umap==0.3.10 numpy==1.18.2 scipy==1.4.1 pandas==1.0.4 scikit-learn==0.22.2.post1 statsmodels==0.12.2 python-igraph==0.9.0 louvain==0.7.0 leidenalg==0.8.0


In [3]:
%load_ext rpy2.ipython

## Paths

In [4]:
path_out = '/home/jovyan/projects/kk14_DCM-lymphoid/results/for_edgeR/210526/'

## Functions

In [5]:
def correlation_analysis(scanpy_object, column_tosplitby, sum_or_mean):
    d = {}
    for cluster_number in np.unique(scanpy_object.obs[column_tosplitby].values):
        #cluster_number=np.unique(heart.obs['louvain_annotated'].values)[1]
        scanpy_object_subset = scanpy_object[scanpy_object.obs[column_tosplitby].isin([cluster_number])]

        if sum_or_mean=="mean":
            d[cluster_number] = np.squeeze(np.asarray(scanpy_object_subset.raw.X.mean(axis=0))) #np.log(+1)
        elif sum_or_mean=="sum":
            d[cluster_number] = np.squeeze(np.asarray(scanpy_object_subset.raw.X.sum(axis=0))) #np.log(+1)
        del scanpy_object_subset
        
    return d

## Read dataset

### read global raw

In [6]:
global_all = sc.read_h5ad('/nfs/team205/kk18/DCM/anndata/global_all_ANNOTATED_CELLSTATES_RAW_V3.h5ad')
global_all

/home/jovyan/my-conda-envs/kk14-env/lib/python3.8/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 881081 × 33538
    obs: 'Barcode', 'Patient', 'Sample', 'Gender', 'Gene', 'Diagnosis', 'Clinical.dominant.mutation', 'Age', 'Mutation.Type', 'Genomic.location', 'Region', 'Origin', 'n_genes', 'percent_mito', 'percent_ribo', 'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo', 'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log', 'predicted_doublets_log', 'Replicate', 'cell_states_20200602', 'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA', 'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921', 'cell_states_long_20200921', 'cell_states_20210218', 'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03', 'leiden_04', 'cell_type', 'cell_states_20210302', 'cell_states_long', 'validated', 'cell_type_toplot1', 'cell_type_toplot2', 'cell_type_toplot3', 'cell_type_toplot4', 'cell_states'
    var: 'gene_ids', 'feature_types', 'genome'

In [7]:
subs = {
    "vEC":"EC",
    "nonvEC":"EC", 
    "SMC":"Mural", 
    "PC":"Mural", 
     } 
# PC-like, EC-like, MP-like, MP_interacting, CM-like, ECM_r, CST3+, ECM_p, Canonical, DAAM1+
x=[subs.get(item,item)  for item in global_all.obs['cell_type']]
global_all.obs['cell_type']=x
pd.value_counts(global_all.obs['cell_type'])/global_all.shape[0]

CM          0.358217
Mural       0.199124
FB          0.176731
EC          0.145108
Myeloid     0.079489
Lymphoid    0.022796
N           0.011461
Mast        0.003926
AD          0.003150
Name: cell_type, dtype: float64

In [8]:
set(global_all.obs[global_all.obs['cell_type']=='Lymphoid']['cell_states'])

{'B',
 'CD4T_act',
 'CD4T_naive',
 'CD4T_reg',
 'CD8T_cytox',
 'CD8T_em',
 'CD8T_te',
 'CD8T_te_IFNGhi',
 'CD8T_trans',
 'ILC',
 'LY0',
 'MAIT-like',
 'NK_CD16hi',
 'NK_CD16hiIFNGhi',
 'NK_CD56hi',
 'Plasma',
 'prolif_Lymphoids',
 'unclassified.1',
 'unclassified.2'}

In [9]:
global_all=global_all[global_all.obs.Region.isin(['RV'])]
global_all=global_all[global_all.obs.validated]
global_all=global_all[global_all.obs['Clinical.dominant.mutation'].isin(['TTN', 'LMNA', 'RBM20', 'PKP2', 'control', 'PVneg'])]
global_all

View of AnnData object with n_obs × n_vars = 182977 × 33538
    obs: 'Barcode', 'Patient', 'Sample', 'Gender', 'Gene', 'Diagnosis', 'Clinical.dominant.mutation', 'Age', 'Mutation.Type', 'Genomic.location', 'Region', 'Origin', 'n_genes', 'percent_mito', 'percent_ribo', 'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo', 'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log', 'predicted_doublets_log', 'Replicate', 'cell_states_20200602', 'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA', 'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921', 'cell_states_long_20200921', 'cell_states_20210218', 'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03', 'leiden_04', 'cell_type', 'cell_states_20210302', 'cell_states_long', 'validated', 'cell_type_toplot1', 'cell_type_toplot2', 'cell_type_toplot3', 'cell_type_toplot4', 'cell_states'
    var: 'gene_ids', 'feature_types', 'genome'

In [10]:
def paste0(i, j, k, l, m):
    return(str(i)+"__"+str(j)+"__"+str(k)+"__"+str(l)+"__"+str(m))


global_all.obs['pool_key']=[paste0(i,j,k,l,m) for i,j,k,l,m in zip(global_all.obs['Clinical.dominant.mutation'],
                                                                   global_all.obs['cell_states'],
                                                                   global_all.obs['Patient'],
                                                                   global_all.obs['X10X_version'],
                                                                   global_all.obs['Gender'])]

Trying to set attribute `.obs` of view, copying.


In [11]:
global_all.raw=global_all
x=correlation_analysis(global_all, 'pool_key', 'sum')
x=pd.DataFrame(x)
x.index=global_all.var_names
x.head()

,LMNA__AD1.0__H06__V3__f,LMNA__AD1.0__H24__V3__f,LMNA__AD1.0__H28__V3__m,LMNA__AD1.1__H06__V3__f,LMNA__AD1.1__H24__V3__f,LMNA__AD1.1__H25__V3__m,LMNA__AD1.1__H28__V3__m,LMNA__AD1.1__H29__V3__f,LMNA__AD2__H06__V3__f,LMNA__AD2__H24__V3__f,...,control__vFB4__BS_H37__V3__m,control__vFB4__D1__V2__f,control__vFB4__D2__V2__m,control__vFB4__D4__V2__f,control__vFB4__D5__V2__f,control__vFB4__D6__V2__m,control__vFB4__D7__V2__m,control__vFB4__H49__V3__f,control__vFB4__H53__V3__m,control__vFB4__H55__V3__m
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.1,1.0,0.0,2.0,2.0,0.0,2.0,8.0,0.0,3.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0
AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
x.to_csv(path_out + "ALL_CELLSTATE_PSEUDOBULK_RV.csv")

## for filter genes

In [13]:
global_all.obs['Gene_Patient']=[str(i) + "_" + str(j) + "_" + str(k) for i,j,k in zip(global_all.obs['Clinical.dominant.mutation'], 
                                                                                      global_all.obs['cell_states'],
                                                                                      global_all.obs['Patient'])]

In [14]:
subset_=sc.AnnData(global_all[:,global_all.var.index].raw.X, # 
                  obs=global_all.obs,
                  var=global_all[:,global_all.var.index].raw.var
                 )
subset_

AnnData object with n_obs × n_vars = 182977 × 33538
    obs: 'Barcode', 'Patient', 'Sample', 'Gender', 'Gene', 'Diagnosis', 'Clinical.dominant.mutation', 'Age', 'Mutation.Type', 'Genomic.location', 'Region', 'Origin', 'n_genes', 'percent_mito', 'percent_ribo', 'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo', 'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log', 'predicted_doublets_log', 'Replicate', 'cell_states_20200602', 'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA', 'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921', 'cell_states_long_20200921', 'cell_states_20210218', 'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03', 'leiden_04', 'cell_type', 'cell_states_20210302', 'cell_states_long', 'validated', 'cell_type_toplot1', 'cell_type_toplot2', 'cell_type_toplot3', 'cell_type_toplot4', 'cell_states', 'pool_key', 'Gene_Patient'
    var: 'gene_ids', 'feature_types', 'genome'

In [15]:
subset_.X.data[:10]

array([1., 1., 1., 1., 1., 2., 1., 1., 1., 1.], dtype=float32)

In [16]:
subset_.raw=subset_
x=correlation_analysis(subset_, 'Gene_Patient', 'mean') # MEan per Gene_Celltype_Patient
x=pd.DataFrame(x)
x.index=global_all.var.index

In [17]:
# del global_all

In [18]:
MUTATIONS=[str(x) + "_" + str(y) for x in global_all.obs['Clinical.dominant.mutation'].unique() for 
           y in global_all.obs['cell_states'].unique()]
MUTATIONS

['control_PC1',
 'control_vCM1.1',
 'control_SMC1.1',
 'control_vCM1.0',
 'control_vCM3.0',
 'control_vCM2',
 'control_vCM4',
 'control_Mast',
 'control_vFB1.0',
 'control_EC1.0',
 'control_SMC2',
 'control_PC2',
 'control_vCM1.3',
 'control_SMC1.2',
 'control_vCM3.1',
 'control_vCM5',
 'control_EC2.0',
 'control_PC3',
 'control_NK_CD56hi',
 'control_vCM1.2',
 'control_MY1',
 'control_vFB1.1',
 'control_NC1.0',
 'control_EC6.0',
 'control_vFB2',
 'control_EC8.0',
 'control_vFB1.2',
 'control_EC7.0',
 'control_vFB4',
 'control_NC2',
 'control_MY7',
 'control_CD8T_cytox',
 'control_CD4T_naive',
 'control_CD8T_trans',
 'control_CD4T_act',
 'control_EC5.0',
 'control_MY12',
 'control_NC1.2',
 'control_MY5',
 'control_MY15',
 'control_NC1.1',
 'control_MY8',
 'control_CD8T_em',
 'control_MY9',
 'control_NK_CD16hi',
 'control_AD2',
 'control_MAIT-like',
 'control_prolif Myeloids',
 'control_NC1.4',
 'control_MY14',
 'control_NC3',
 'control_NK_CD16hiIFNGhi',
 'control_CD16+ Mo',
 'control_MY

In [19]:
pre=[]
for MUTATION in MUTATIONS:
    OUT=np.where([re.match(MUTATION, COLNAME) for COLNAME in x.columns])[0]
    pre.append(OUT)
pre

[array([2376, 2377, 2378, 2379, 2380, 2381, 2382, 2383, 2384, 2385, 2386,
        2387, 2388, 2389, 2390]),
 array([2540, 2541, 2542, 2543, 2544, 2545, 2546, 2547, 2548, 2549, 2550,
        2551, 2552, 2553, 2554]),
 array([2429, 2430, 2431, 2432, 2433, 2434, 2435, 2436, 2437, 2438, 2439,
        2440, 2441, 2442, 2443]),
 array([2525, 2526, 2527, 2528, 2529, 2530, 2531, 2532, 2533, 2534, 2535,
        2536, 2537, 2538, 2539]),
 array([2600, 2601, 2602, 2603, 2604, 2605, 2606, 2607, 2608, 2609, 2610,
        2611, 2612, 2613, 2614]),
 array([2585, 2586, 2587, 2588, 2589, 2590, 2591, 2592, 2593, 2594, 2595,
        2596, 2597, 2598, 2599]),
 array([2630, 2631, 2632, 2633, 2634, 2635, 2636, 2637, 2638, 2639, 2640,
        2641, 2642, 2643, 2644]),
 array([2233, 2234, 2235, 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243,
        2244, 2245, 2246]),
 array([2660, 2661, 2662, 2663, 2664, 2665, 2666, 2667, 2668, 2669, 2670,
        2671, 2672, 2673, 2674]),
 array([1973, 1974, 1975, 1976, 19

In [20]:
from fastprogress.fastprogress import  progress_bar

In [21]:
LIST_COLNAMES=[]
for MUTATION in progress_bar(MUTATIONS):
    otherMUTATIONS = list(set(MUTATIONS) - set([MUTATION]))
    OUT=np.array([i for i in range(len(x.columns)) \
                  if (MUTATION in x.columns[i]) & \
                     (all([mut not in x.columns[i] for mut in otherMUTATIONS if mut not in MUTATION])) # for specificity (e.g. for 'B' and 'B_plasma')
                 ]
                )
    LIST_COLNAMES.append(OUT)

x=[x.iloc[:,i].mean(axis=1) for i in LIST_COLNAMES] # Mean per Genotype_Cellstate
x=pd.concat(x, axis=1)

In [22]:
len(LIST_COLNAMES)

474

In [23]:
for i in range(len(LIST_COLNAMES)):
    if len(pre[i]) == len(LIST_COLNAMES[i]):
        if all(LIST_COLNAMES[i]==pre[i]):
            pass
        else:
            print(MUTATIONS[i])
    else:
        print(MUTATIONS[i])

control_MY1
control_NK_CD16hi
control_CD16+ Mo
control_CD8T_te
TTN_MY1
TTN_NK_CD16hi
TTN_CD16+ Mo
TTN_CD8T_te
PVneg_MY1
PVneg_NK_CD16hi
PVneg_CD16+ Mo
PVneg_CD8T_te
LMNA_MY1
LMNA_NK_CD16hi
LMNA_CD16+ Mo
LMNA_CD8T_te
RBM20_MY1
RBM20_NK_CD16hi
RBM20_CD16+ Mo
RBM20_CD8T_te
PKP2_MY1
PKP2_NK_CD16hi
PKP2_CD16+ Mo
PKP2_CD8T_te


In [24]:
pre[i]

array([687, 688, 689, 690])

In [25]:
LIST_COLNAMES[i]

array([687, 688, 689, 690])

In [26]:
len(pre[0])

15

In [27]:
x.columns=MUTATIONS
x.head()

,control_PC1,control_vCM1.1,control_SMC1.1,control_vCM1.0,control_vCM3.0,control_vCM2,control_vCM4,control_Mast,control_vFB1.0,control_EC1.0,...,PKP2_CD8T_te_IFNGhi,PKP2_AD1.0,PKP2_cDC1,PKP2_MY16,PKP2_Meso,PKP2_MY10,PKP2_prolif_Lymphoids,PKP2_unclassified.2,PKP2_NC1.3,PKP2_NC6
MIR1302-2HG,0.000000,0.00000,0.000000,0.000028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0
FAM138A,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0
OR4F5,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0
AL627309.1,0.004966,0.01577,0.003131,0.015781,0.022191,0.025318,0.020915,0.007937,0.007215,0.002436,...,0.001825,0.0,0.0,0.0,0.0,0.02306,0.0,0.0,0.044444,0.0
AL627309.3,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0


In [28]:
x.to_csv(path_out + "ALL_CELLSTATE_PSEUDOBULK_FILTERING_RV.csv")

In [29]:
x = pd.Series(global_all.obs.cell_type.values,
          index=global_all.obs.cell_states).to_dict()
x=pd.DataFrame.from_dict(x, orient='index')
x[x[0]=='Lymphoid']

,0
NK_CD56hi,Lymphoid
CD8T_cytox,Lymphoid
CD4T_naive,Lymphoid
CD8T_trans,Lymphoid
CD4T_act,Lymphoid
CD8T_em,Lymphoid
NK_CD16hi,Lymphoid
MAIT-like,Lymphoid
NK_CD16hiIFNGhi,Lymphoid
unclassified.1,Lymphoid


In [30]:
x.to_csv(path_out + "CELLSTATE_TRANSLATION_TABLE_RV.csv")

## Pseudobulk for cell_type 

In [31]:
global_all.obs.drop(['pool_key'], axis=1, inplace=True)
global_all.obs.head()

,Barcode,Patient,Sample,Gender,Gene,Diagnosis,Clinical.dominant.mutation,Age,Mutation.Type,Genomic.location,...,cell_type,cell_states_20210302,cell_states_long,validated,cell_type_toplot1,cell_type_toplot2,cell_type_toplot3,cell_type_toplot4,cell_states,Gene_Patient
19122,AAACCCAAGCGTGTCC-BS_H15_RV_premrna,BS_H15,BS_H15_RV0_premrna,f,control,Control,control,54.0,nan,nan,...,Mural,PC1,PC1_vent,True,PC,PC,PC,PC,PC1,control_PC1_BS_H15
19123,AAACCCAAGGGCTAAC-BS_H15_RV_premrna,BS_H15,BS_H15_RV0_premrna,f,control,Control,control,54.0,nan,nan,...,CM,vCM1.3,vCM1.3,True,CM,CM,CM,CM,vCM1.1,control_vCM1.1_BS_H15
19124,AAACCCAAGTAACCTC-BS_H15_RV_premrna,BS_H15,BS_H15_RV0_premrna,f,control,Control,control,54.0,nan,nan,...,Mural,PC1,PC1_vent,True,PC,PC,PC,PC,PC1,control_PC1_BS_H15
19125,AAACCCAAGTGCCCGT-BS_H15_RV_premrna,BS_H15,BS_H15_RV0_premrna,f,control,Control,control,54.0,nan,nan,...,Mural,SMC1.1,SMC1.1_basic,True,SMC,SMC,SMC,SMC,SMC1.1,control_SMC1.1_BS_H15
19126,AAACCCACAAGTGGAC-BS_H15_RV_premrna,BS_H15,BS_H15_RV0_premrna,f,control,Control,control,54.0,nan,nan,...,CM,vCM1.0,vCM1.0,True,CM,CM,CM,CM,vCM1.0,control_vCM1.0_BS_H15


In [32]:
def paste0(i, j, k, l, m):
    return(str(i)+"__"+str(j)+"__"+str(k)+"__"+str(l)+"__"+str(m))


global_all.obs['pool_key']=[paste0(i,j,k,l,m) for i,j,k,l,m in zip(global_all.obs['Clinical.dominant.mutation'],
                                                                   global_all.obs['cell_type'],
                                                                   global_all.obs['Patient'],
                                                                   global_all.obs['X10X_version'],
                                                                   global_all.obs['Gender'])]

In [33]:
x=correlation_analysis(global_all, 'pool_key', 'sum')
x=pd.DataFrame(x)
x.index=global_all.var.index
x.head()

,LMNA__AD__H06__V3__f,LMNA__AD__H24__V3__f,LMNA__AD__H25__V3__m,LMNA__AD__H27__V3__f,LMNA__AD__H28__V3__m,LMNA__AD__H29__V3__f,LMNA__CM__H06__V3__f,LMNA__CM__H24__V3__f,LMNA__CM__H25__V3__m,LMNA__CM__H27__V3__f,...,control__N__BS_H37__V3__m,control__N__D1__V2__f,control__N__D2__V2__m,control__N__D4__V2__f,control__N__D5__V2__f,control__N__D6__V2__m,control__N__D7__V2__m,control__N__H49__V3__f,control__N__H53__V3__m,control__N__H55__V3__m
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.1,6.0,1.0,2.0,1.0,11.0,0.0,62.0,30.0,53.0,25.0,...,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
x.to_csv(path_out + "ALL_CELLTYPE_PSEUDOBULK_RV.csv")

## Abundance for cell_type

In [35]:
x=pd.crosstab(global_all.obs['cell_type'],global_all.obs['Patient'])
y=pd.crosstab(global_all.obs['cell_states'],global_all.obs['Patient'])

In [36]:
x.to_csv(path_out + "ABSOLUTE_CELLTYPE_NUMBER_RV.csv")
y.to_csv(path_out + "ABSOLUTE_CELLSTATES_NUMBER_RV.csv")

In [37]:
y

Patient,BS_H15,BS_H20,BS_H25,BS_H26,BS_H35,BS_H37,D1,D2,D4,D5,...,H33,H34,H35,H36,H39,H41,H49,H53,H55,IC_H04
cell_states,,,,,,,,,,,,,,,,,,,,,
AD1.0,0,41,0,42,6,0,10,2,36,3,...,54,0,0,1,0,0,67,9,4,14
AD1.1,0,2,0,24,11,0,0,0,1,0,...,5,0,0,0,0,0,0,0,0,9
AD2,1,34,6,23,7,0,0,0,1,0,...,5,0,0,0,1,1,30,10,0,3
AD3,0,3,4,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B,0,0,1,0,4,1,0,0,0,0,...,10,1,1,0,3,1,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vFB1.1,15,7,11,45,38,15,232,60,268,1,...,42,11,24,98,18,156,141,101,2,0
vFB1.2,6,1,17,18,40,12,12,65,66,32,...,38,54,14,16,20,39,20,14,39,63
vFB2,16,5,18,11,43,7,5,33,156,25,...,184,161,228,277,45,280,11,54,197,37
